In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
pd.options.display.float_format = '{:.4f}'.format
from itertools import combinations
from sklearn.metrics import mean_squared_error,r2_score,root_mean_squared_error
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA as ARIMA
import statsmodels.api as sm
import statsmodels.tsa.api as smt
import math
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)
# Load the data

path='Datasets/GPM_3IMERGDL_06_precipitationCal.20000601-20230531.algeria.csv'
data = pd.read_csv('./{} '.format(path))
list_of_column_names = []
for row in data:
    # adding the first row
    list_of_column_names.append(row)
 
    # breaking the loop after the first iteration itself

#function that test dataset stationarity by plotting rolling mean and standard deviation and perform augmented Dickey-Fuller test

def stationarity_test(timeseries):
    #Determing rolling statistics
    rolling_mean = timeseries.rolling(window=365).mean()
    rolling_std = timeseries.rolling(window=365).std()

    #Plot rolling statistics:
    plt.figure(figsize=(15,5))
    original = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolling_mean, color='red', label='Rolling Mean')
    standard_deviation = plt.plot(rolling_std, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)

    #Perform augmented Dickey-Fuller test:
    print('Results of Dickey-Fuller Test:')
    dickeyFuller_test = adfuller(timeseries, autolag='AIC')
    dickeyFuller_output = pd.Series(dickeyFuller_test[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dickeyFuller_test[4].items():
        dickeyFuller_output['Critical Value (%s)'%key] = value
    print(dickeyFuller_output)


#function that plots the autocorrelation and partial autocorrelation of the time series
def stationarity_test_plot(y, lags=None, figsize=(12, 7), style='bmh'):
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
        
    with plt.style.context(style):    
        fig = plt.figure(figsize=figsize)
        layout = (2, 2)
        ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1, 0))
        pacf_ax = plt.subplot2grid(layout, (1, 1))
        
        y.plot(ax=ts_ax)
        p_value = sm.tsa.stattools.adfuller(y)[1]
        ts_ax.set_title('Time Series Analysis Plots\n Dickey-Fuller: p={0:.4f}'.format(p_value))
        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax)
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax)
        plt.tight_layout()




In [ ]:

# Data architecture

data.info()


In [ ]:
# Show the data

data.head(-1)


In [ ]:

# Check for NULL values

sns.heatmap(data.isnull(),cmap = 'magma',cbar = False)


In [ ]:
# Data description

data.describe()

In [ ]:
# Converting the datatype of the Day column to datetime datatype and setting it as the index of the dataset

data['day'] = pd.to_datetime(data['time'])
data = data.drop(columns = 'time')
data = data.set_index('day')

# Renaming the column Month to Date and mean_TRMM_3B42RT_Daily_7_precipitation to precipitation as well.

data = data.rename(columns = {'{}'.format(list_of_column_names[1]):'precipitation'})
data.head(-1)

In [ ]:
# use moving average method to smooth data
data['moving_average'] = data['precipitation'].rolling(window=3,min_periods=0).mean()

# plot the difference between original data and smoothed data
fig,ax = plt.subplots(nrows = 1,ncols = 1,figsize = (15,5))

plt.subplot(1,1,1)
plt.plot(data['precipitation'],label = 'Original Data')
plt.plot(data['moving_average'],label = 'Smoothed Data')
plt.xlabel('Time')
plt.ylabel('Precipitation (mm per day)')
plt.legend(loc="upper left")
plt.title('Original Data / Smoothed Data Comparison')

In [ ]:
# plot seasonal decomposition (original , trend, seasonal, residual) to check the trend and seasonality of the data and to check if the data is stationary or not

plt.figure(figsize = (15,5))
data['moving_average'].plot()
decomposition = sm.tsa.seasonal_decompose(data['moving_average'],period=365).plot()
plt.show()


In [ ]:
# test if data is stationary or not
stationarity_test(data['moving_average'])

#plot acf and pacf to chose teh right parametres for ARIMA model
stationarity_test_plot(data['moving_average'],lags=50)


p-value is < 0.05 & Critical Value (5%) is > test static so we conclude that the data is stationary


In [ ]:

#implementing arima model to our smoothed data with parameters p=2,d=0,q=1 that are brought from acf and pacf graphs
p=2
d=0
q=2
arima_model = ARIMA(data['moving_average'],order = (p,d,q))
arima_model_fit = arima_model.fit()
print(arima_model_fit.summary())

#devide the data into train and test set 

testSize=730  # you can chose forecasting for 1 year (365), 2 years (730) or 3 years(1095)
size = int(len(data['moving_average']) - testSize)
train, test = data['moving_average'][:size], data['moving_average'][size:]

print('\t ARIMA MODEL : In- Sample Forecasting \n')

#start forecasting

training_data  = [x for x in train]
predicted_list = []
for t in range(len(test)):
    
    arima_model = ARIMA(training_data, order=(p,d,q))
    arima_model_fit = arima_model.fit()
    output = arima_model_fit.forecast()
    predicted_value = output[0]
    predicted_list.append(float(predicted_value))
    original_value = test[t]
    training_data.append(original_value)
    print('predicted value= %f, expected value= %f' % (predicted_value, original_value))


In [ ]:
#plot the orginal and predicted values

predictions_series = pd.Series(predicted_list, index = test.index)
fig,ax = plt.subplots(nrows = 1,ncols = 1,figsize = (15,5))

plt.subplot(1,1,1)
plt.plot(data['moving_average'][size:len(data)],label = 'Original Precipitation Values')
plt.plot(predictions_series,label = 'Predicted Precpitation Values')
plt.xlabel('Time')
plt.ylabel('Precipitation (mm per day)')
plt.legend(loc="upper left")
plt.title('ARIMA Model ('+ str(p)+' '+ str(d)+' '+ str(q)+') With Data Smoothing:'+ str(testSize) +' days ('+ str(int(testSize/365)) +' years)  In-Sample  Forecasting')
plt.show()
# print succes scores


RMSE = math.sqrt(mean_squared_error(test,predicted_list))
R2 = r2_score(test, predicted_list)
print('Test RMSE: %.4f' % RMSE)
print('r2 score: %.4f'% R2)